In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import pickle

In [3]:
train_dir = "/content/drive/MyDrive/Cattle Classification/data/train"
val_dir = "/content/drive/MyDrive/Cattle Classification/data/val"
test_dir = "/content/drive/MyDrive/Cattle Classification/data/test"

IMG_SIZE = 224

train_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    rotation_range = 15,
    horizontal_flip = True,
    width_shift_range=0.10,
    height_shift_range=0.10,
    zoom_range = 0.2,
    fill_mode = "nearest"
)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = 32,
    class_mode = "categorical",
    shuffle = True
)

val_datagen = ImageDataGenerator(preprocessing_function= preprocess_input)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = 32,
    class_mode = "categorical",
    shuffle = False
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = 32,
    class_mode = "categorical",
    shuffle = False
)

base_model = ResNet50(include_top=False, weights="imagenet", input_shape=(IMG_SIZE, IMG_SIZE, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.4)(x)
output = Dense(5, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer = Adam(learning_rate=0.001),
    loss = "categorical_crossentropy",
    metrics = ["accuracy"]
)

if __name__ == "__main__":

  history = model.fit(
    train_data,
    validation_data = val_data,
    epochs = 15,
    verbose = 1
  )

  model.save("/content/drive/MyDrive/Cattle Classification/saved_models/resnet50.h5")

  with open("/content/drive/MyDrive/Cattle Classification/saved_models/history_resnet50.pkl", "wb") as f:
    pickle.dump(history.history, f)

test_loss, test_acc = model.evaluate(test_data, verbose=1)
print(f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")

Found 965 images belonging to 5 classes.
Found 119 images belonging to 5 classes.
Found 124 images belonging to 5 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 351s 11s/step - accuracy: 0.4394 - loss: 1.6829 - val_accuracy: 0.7395 - val_loss: 0.6653
Epoch 2/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 16s 505ms/step - accuracy: 0.7379 - loss: 0.7838 - val_accuracy: 0.8067 - val_loss: 0.5732
Epoch 3/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 16s 506ms/step - accuracy: 0.8131 - loss: 0.5644 - val_accuracy: 0.8739 - val_loss: 0.4807
Epoch 4/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 16s 504ms/step - accuracy: 0.8448 - loss: 0.4696 - val_accuracy: 0.8487 - val_loss: 0.4594
Epoch 5/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 16s 505ms/step - accuracy: 0.8551 - loss: 0.3894 - val_accuracy: 0.8487 - val_loss: 0.3999
Epoch 6/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 17s 541ms/step - accuracy: 0.8768 - loss: 0.3452 - val_accuracy: 0.8571 - val_loss: 0.3808
Epoch 7/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 16s 504ms/step - accuracy: 0.8828 - loss: 0.3718 - val_accuracy: 0.8824 - val_loss: 0.3855
Epoch 8/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 16s 501ms/step - accuracy: 0.9126 - loss: 0.2882 - val_accur

4/4 ━━━━━━━━━━━━━━━━━━━━ 33s 11s/step - accuracy: 0.8294 - loss: 0.5756
Test Loss: 0.4704 | Test Acc: 0.8548
